In [ ]:
%matplotlib inline
import sys
sys.path.append('/Volumes/SANDISK128/Documents/Thesis/Python/')
import netCDF4
import numpy as np
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import datetime

import createFolder as cF


In [ ]:
def create_netCDF_Haukeli(year,month,day,stn_lat,stn_lon,var_name,time_shape,ens_memb):
    thredds  = 'http://thredds.met.no/thredds/dodsC/meps25epsarchive'
    time = '00'
    fn = netCDF4.Dataset('%s/%s/%s/%s/meps_full_2_5km_%s%s%sT%sZ.nc' %(thredds,year,month,day,year,month,day,time),'r')


## Latitudes
## [y = 949][x = 739]
    latitude = fn.variables['latitude']

## Longitudes 
## [y = 949][x = 739]
    longitude = fn.variables['longitude']

## Time
    tid = fn.variables['time']
    
## hybrid levels atmosphere_hybrid_sigma_pressure_coordinate
    hybrid = fn.variables['hybrid']
### From the with Vertical Levels ( height0 ) : 0.0 m

## 1. air_temperature_0m = Surface temperature (T0M)
## [time = 56][height0 = 1][ensemble_member = 10][y = 949][x = 739]
#var_name = 'air_temperature_0m'
    temp_sfc = fn.variables[var_name]    # [K]


# Stations latitude longitude, the nearest gridpoint we are looking for
#stn_lat = 59.8
#stn_lon = 7.2

# Now find the absolute value of the difference between the  station's lat/lon with every point in the grid. 
# This tells us how close a point is to the particular latitude and longitude.

    abslat = np.abs(latitude[:,:]-stn_lat)
    abslon= np.abs(longitude[:,:]-stn_lon)

# Now we need to combine these two results. We will use numpy.maximum, which takes two arrays and finds the local 
# maximum.
    c = np.maximum(abslon, abslat)

# If you don't like flattened arrays, you can also get the row/column index like this
    y, x = np.where(c == np.min(c))

### fill values of missing data
    if np.ma.is_masked(temp_sfc[:,:,ens_memb,y[0],x[0]]):
        mask = temp_sfc[:,:,ens_memb,y[0],x[0]].mask
        fill_value = temp_sfc[:,:,ens_memb,y[0],x[0]].fill_value
        marr = np.ma.array(temp_sfc[:,:,ens_memb,y[0],x[0]], mask = mask, fill_value = fill_value)
        dt_var = marr.filled().dtype
    else: 
        fill_value = -99999
        marr = temp_sfc[:,:,ens_memb,y[0],x[0]]
        dt_var = marr.dtype
    
    dt_tid = tid.dtype
    dt_hyb = hybrid.dtype



### write netCDF file 
    if ens_memb < 9:
        ens_memb = '0%s' %(str(ens_memb+1))
    else:
        ens_memb = ens_memb+1

    f_sfc = netCDF4.Dataset('../../dataMEPS/%s/%s%s%s_%s.nc' %(var_name,year,month,day,ens_memb), 'w')
    f_sfc.createDimension('time', time_shape)            # [time = 67]
    f_sfc.createDimension('hybrid', 65)          # [hybrid = 65]
    f_sfc.createDimension('ensemble_member', 1)  # [ensemble_member = 1]
    f_sfc.createDimension('y', 1)                # [y = 1]
    f_sfc.createDimension('x', 1)                # [x = 1]

    t = f_sfc.createVariable('time', dt_tid, ('time'),zlib = True)
    hyb = f_sfc.createVariable('hybrid_height', dt_hyb, ('hybrid'), zlib = True)
    grid_temp_sfc = f_sfc.createVariable('%s' %(var_name), dt_var, ('time','hybrid','ensemble_member','y','x'),fill_value = fill_value,zlib=True)



    t[:] = tid[:]
    hyb[:] = hybrid[:]
    grid_temp_sfc[:] = marr

    f_sfc.close()

In [ ]:
# Stations latitude longitude, the nearest gridpoint we are looking for
stn_lat = 59.8
stn_lon = 7.2

year = '2016'
month = '12'

# Variable
var_name = [ \
    ### From with Vertical Levels ( hybrid )
#            'air_temperature_ml',                           # [K]                   
 #           'atmosphere_cloud_condensed_water_content_ml',  # [kg/m^2]
 #           'atmosphere_cloud_ice_content_ml',              # [kg/m^2]
  #          'graupelfall_amount_ml',                        # [kg/m^2]
   #         'pressure_departure',                           # [Pa]
    #        'rainfall_amount_ml',                           # [kg/m^2]
            'snowfall_amount_ml'#,                           # [kg/m^2]
      #      'specific_humidity_ml'                          # [kg/kg]
           ]                         

    
# 16th doesnt exist, the other days have different shapes
t = [
    1, 2, 3, 
    4, 5, 6 , 
    7, 8, 9,10, 11,
    12,13,14,15,    
    17,18,19,20, 
    21,22,23,
    24,25,26,
    27,28,29,30, 
    31
    ]




for name in var_name:
    cF.createFolder('../../dataMEPS/%s/' %(name))
    # Date
    for day in t:
        if day < 10:
            day = '0%s' %(str(day))
        if (day == 17):
            time_shape = 60
        elif (day == 18):
            time_shape = 56
        elif (day == 19):
            time_shape = 53
        else:
            time_shape = 67
        day = str(day)
        for ens_memb in range(0,10):
            create_netCDF_Haukeli(year,month,day,stn_lat,stn_lon,name,time_shape,ens_memb)
            print('File written: %s %s%s%s_0%s.nc' %(name,year,month,day,(ens_memb+1)))